In [1]:
import json
import pandas as pd
from datasets import load_from_disk


data = json.load(open('../data/raw/wikipedia_documents.json'))
wiki = pd.DataFrame(data).T

dataset = load_from_disk("../data/raw/train_dataset/")
train_df = pd.DataFrame(dataset['train'])
valid_df = pd.DataFrame(dataset['validation'])

mrc = pd.concat([train_df, valid_df])

wiki.shape, mrc.shape

/opt/conda/envs/llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


((60613, 8), (4192, 7))

In [2]:
wiki.head(3)

,text,corpus_source,url,domain,title,author,html,document_id
0,"이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개...",위키피디아,TODO,None,나라 목록,None,None,0
1,이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 1장을 참고로 하였다. 협정...,위키피디아,TODO,None,나라 목록,None,None,1
2,현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신이다. 친...,위키피디아,TODO,None,백남준,None,None,2


In [3]:
# make new list that contains all wiki['text']
wiki_text = wiki['text'].tolist()
len(wiki_text)

60613

In [4]:
mrc.head(3)

,title,context,question,id,answers,document_id,__index_level_0__
0,미국 상원,미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?,mrc-1-000067,"{'answer_start': [235], 'text': ['하원']}",18293,42
1,인사조직관리,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,현대적 인사조직관리의 시발점이 된 책은?,mrc-0-004397,"{'answer_start': [212], 'text': ['《경영의 실제》']}",51638,2873
2,강희제,강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?,mrc-1-000362,"{'answer_start': [510], 'text': ['백성']}",5028,230


In [5]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("intfloat/e5-mistral-7b-instruct")

Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.47s/it]


In [6]:
import torch

# cache the model on the GPU to reduce GPU memory usage
torch.cuda.empty_cache()

# In case you want to reduce the maximum sequence length:
model.max_seq_length = 4096

# queries = [
#     "how much protein should a female eat",
#     "summit define",
# ]
# documents = [
#     "As a general guideline, the CDC's average requirement of protein for women ages 19 to 70 is 46 grams per day. But, as you can see from this chart, you'll need to increase that if you're expecting or training for a marathon. Check out the chart below to see how much protein you should be eating each day.",
#     "Definition of summit for English Language Learners. : 1  the highest point of a mountain : the top of a mountain. : 2  the highest level. : 3  a meeting or series of meetings between the leaders of two or more governments."
# ]

# x = 5
# queries, documents = [], []
# for i in range(x):
#     queries.append(mrc['question'].iloc[i])
#     documents.append(mrc['context'].iloc[i])


queries = [mrc['question'].iloc[0], mrc['question'].iloc[1]]
documents = wiki['text'].tolist()

query_embeddings = model.encode(queries, prompt_name="web_search_query")
document_embeddings = model.encode(documents)

scores = (query_embeddings @ document_embeddings.T) * 100

top_k = 1
for i in range(len(queries)):
    print(f"Query: {queries[i]}")
    top_indices = np.argsort(scores[i])[::-1][:top_k]  # 유사도 상위 3개 인덱스
    for rank, idx in enumerate(top_indices, 1):
        print(f"Top {rank} Document {idx}: {documents[idx]}")
        print(f"Score: {scores[i][idx]:.2f}")
    print()

OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 GiB. GPU 0 has a total capacity of 31.74 GiB of which 860.38 MiB is free. Process 3030908 has 30.90 GiB memory in use. Of the allocated memory 30.51 GiB is allocated by PyTorch, and 16.81 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [9]:
torch.cuda.empty_cache()

In [3]:
wiki['text_length'] = wiki['text'].apply(len)
wiki['text_length'].describe()

count    60613.000000
mean       755.565044
std        762.962671
min        184.000000
25%        414.000000
50%        577.000000
75%        857.000000
max      46099.000000
Name: text_length, dtype: float64

In [4]:
mrc['question_length'] = mrc['question'].apply(len)
mrc['question_length'].describe()

count    4192.000000
mean       29.315124
std         8.726480
min         8.000000
25%        23.000000
50%        29.000000
75%        35.000000
max        78.000000
Name: question_length, dtype: float64

In [9]:
# load csv: data/experiments/query_topk_results.csv
query_topk_results = pd.read_csv('../data/experiments/query_topk_results.csv')
query_topk_results.head()

,query,answer_text,top1_text,top1_score,top2_text,top2_score,top3_text,top3_score
0,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?,미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,75.96260,"이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개...",69.189575,이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 1장을 참고로 하였다. 협정...,68.859350
1,현대적 인사조직관리의 시발점이 된 책은?,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,85.56025,현대에 들어와 데이터 과학자들로 구성된 통계 조직은 기관과 단체 그리고 기업의 수익...,70.729550,켄 톰슨(Ken Thompson) 등과 함께 최초의 유닉스(Unix) 시스템을 개발...,69.725945
